In [ ]:
import pandas as pd
import re
from urllib.parse import urlparse, urlunparse, parse_qs, urlencode

INPUT_CSV = "merged original mentions from altmetric.csv"
OUT_CLEAN_CSV = "mentions_original_cleaned.csv"

df = pd.read_csv(
    INPUT_CSV,
    dtype=str,
    keep_default_na=False,
    na_values=[]
)

df.columns = [c.strip() for c in df.columns]

keep_types = {"news story", "news", "blog post", "blog"}
if "Mention Type" in df.columns:
    mask = df["Mention Type"].str.lower().isin(keep_types)
    df = df[mask].copy()

if "Mention URL" in df.columns:
    URL_COL = "Mention URL"
else:
    url_like = [c for c in df.columns if "url" in c.lower()]
    URL_COL = url_like[0] if url_like else None

if URL_COL is None:
    df["Mention URL"] = ""
else:
    df["Mention URL"] = df[URL_COL].astype(str).str.strip()

df.loc[df["Mention URL"].str.lower().eq("nan"), "Mention URL"] = ""

def normalize_url(u: str) -> str:
    if not isinstance(u, str):
        return ""
    s = u.strip()
    if not s:
        return ""
    try:
        uo = urlparse(s)
        if uo.scheme.lower() not in {"http", "https"}:
            return ""
        q = parse_qs(uo.query, keep_blank_values=True)
        for k in list(q.keys()):
            lk = k.lower()
            if lk.startswith("utm_") or lk in {"fbclid", "gclid"}:
                q.pop(k, None)
        new_q = urlencode(q, doseq=True)
        path = uo.path[:-1] if uo.path.endswith("/") else uo.path
        return urlunparse((uo.scheme.lower(), uo.netloc.lower(), path, uo.params, new_q, ""))
    except Exception:
        return ""

df["norm_url"] = df["Mention URL"].map(normalize_url)

dropped_empty = df[df["norm_url"] == ""].copy()
kept = df[df["norm_url"] != ""].copy()

dedup_keys = ["norm_url"]
for c in ["Details Page URL", "DOI", "PubMed ID", "ArXiv ID", "Patent Number", "Mention Type"]:
    if c in kept.columns:
        dedup_keys.append(c)

sort_cols = ["Mention Date"] if "Mention Date" in kept.columns else dedup_keys
mentions_clean = (kept.sort_values(sort_cols)
                       .drop_duplicates(subset=dedup_keys)
                       .reset_index(drop=True))

mentions_clean.to_csv(OUT_CLEAN_CSV, index=False)

print("=== Summary ===")
print(f"Total rows in CSV:           {len(df)}")
print(f"Rows dropped (empty URL):    {len(dropped_empty)}")
print(f"Rows kept (valid URL):       {len(kept)}")
print(f"Cleaned mentions (pairwise): {len(mentions_clean)} -> {OUT_CLEAN_CSV}")
print("Dedup keys:", dedup_keys)

In [ ]:
import pandas as pd
import re
from urllib.parse import urlparse, urlunparse, parse_qs, urlencode

INPUT_CSV = "mentions_retraction_cleaned.csv"
OUT_CLEAN_CSV = "merged retraction mentions from altmetric.csv"

df = pd.read_csv(
    INPUT_CSV,
    dtype=str,
    keep_default_na=False,
    na_values=[]
)

df.columns = [c.strip() for c in df.columns]

keep_types = {"news story", "news", "blog post", "blog"}
if "Mention Type" in df.columns:
    mask = df["Mention Type"].str.lower().isin(keep_types)
    df = df[mask].copy()

if "Mention URL" in df.columns:
    URL_COL = "Mention URL"
else:
    url_like = [c for c in df.columns if "url" in c.lower()]
    URL_COL = url_like[0] if url_like else None

if URL_COL is None:
    df["Mention URL"] = ""
else:
    df["Mention URL"] = df[URL_COL].astype(str).str.strip()

df.loc[df["Mention URL"].str.lower().eq("nan"), "Mention URL"] = ""

def normalize_url(u: str) -> str:
    if not isinstance(u, str):
        return ""
    s = u.strip()
    if not s:
        return ""
    try:
        uo = urlparse(s)
        if uo.scheme.lower() not in {"http", "https"}:
            return ""
        q = parse_qs(uo.query, keep_blank_values=True)
        for k in list(q.keys()):
            lk = k.lower()
            if lk.startswith("utm_") or lk in {"fbclid", "gclid"}:
                q.pop(k, None)
        new_q = urlencode(q, doseq=True)
        path = uo.path[:-1] if uo.path.endswith("/") else uo.path
        return urlunparse((uo.scheme.lower(), uo.netloc.lower(), path, uo.params, new_q, ""))
    except Exception:
        return ""

df["norm_url"] = df["Mention URL"].map(normalize_url)

dropped_empty = df[df["norm_url"] == ""].copy()
kept = df[df["norm_url"] != ""].copy()

dedup_keys = ["norm_url"]
for c in ["Details Page URL", "DOI", "PubMed ID", "ArXiv ID", "Patent Number", "Mention Type"]:
    if c in kept.columns:
        dedup_keys.append(c)

sort_cols = ["Mention Date"] if "Mention Date" in kept.columns else dedup_keys
mentions_clean = (kept.sort_values(sort_cols)
                       .drop_duplicates(subset=dedup_keys)
                       .reset_index(drop=True))

mentions_clean.to_csv(OUT_CLEAN_CSV, index=False)

print("=== Summary ===")
print(f"Total rows in CSV:           {len(df)}")
print(f"Rows dropped (empty URL):    {len(dropped_empty)}")
print(f"Rows kept (valid URL):       {len(kept)}")
print(f"Cleaned mentions (pairwise): {len(mentions_clean)} -> {OUT_CLEAN_CSV}")
print("Dedup keys:", dedup_keys)